In [75]:
import pandas as pd
import numpy as np
import calendar

def read_csv_data(filepath, rows=None, encoding="unicode_escape"):
    return pd.read_csv(filepath, encoding="utf-8-sig", sep=",", nrows=rows)

def extract_month_data(df, month_year_str):
    """Extract data for a specific month."""
    return df[df["Timestamp"].str.contains(month_year_str)]

def combine_monthly_data(bedroom_df, lr_df, pmv_df, month_str):
    x = extract_month_data(bedroom_df, month_str)
    y = extract_month_data(lr_df, month_str)
    
    # Rename columns for clarity
    x.columns = ['bedroom_' + str(col) for col in x.columns]
    y.columns = ['living_room_' + str(col) for col in y.columns]
    
    combined_df = pd.concat([x, y], axis=1).fillna(0)
    
    # Drop the Timestamp columns
    combined_df = combined_df.drop(columns=[col for col in combined_df if 'Timestamp' in col])
    
    # Get the TSV column name for the given month/year
    month_name = month_str.split("/")[0]
    year_suffix = month_str.split("/")[1][-2:]
    month_name_dict = {
        "11": "November",
        "12": "December",
        "01": "January",
        "02": "February",
        "03": "March"
    }
    month_col_name = month_name_dict[month_name] + year_suffix + "-TSV"
    
    # Create a dictionary to hold the TSV values for each ID for the given month/year
    tsv_values = pmv_df.set_index('ID')[month_col_name].to_dict()
    
    # Create a new row filled with NaN, except for the ID columns which will hold the TSV values
    new_row = {col: np.nan for col in combined_df.columns}
    for col in combined_df.columns:
        if "bedroom_ID" in col or "living_room_ID" in col:
            id_str = col.split('ID')[-1]
            id_num = int(id_str)
            new_row[col] = tsv_values.get(id_num, np.nan)

    
    combined_df = pd.concat([combined_df, pd.DataFrame([new_row])], ignore_index=True)

    
    # Save the modified combined_df
    combined_df.to_csv(month_str.replace("/", "_") + ".csv", index=False, encoding='utf-8-sig')

# Rest of the main function...








def process_pm_data(df, year_suffix):
    """Process PMV Data."""
    # Identify TSV columns
    tsv_cols = [col for col in df.columns if "TSV" in col]
    
    # Rename columns for clarity and month-year format
    rename_dict = {}
    for col in tsv_cols:
        month_name = col.split(' ')[0]
        rename_dict[col] = month_name + year_suffix + "-TSV"  # Removed the underscore before the year_suffix
    
    return df.rename(columns=rename_dict)


def main():
    # Load Data
    bedroom_df = read_csv_data("Winter_thermal_comfort_dataset/Bedroom_data.csv", 4007).drop("ID27", axis=1)
    lr_df = read_csv_data("Winter_thermal_comfort_dataset/living_room_data.csv", 4007, encoding="utf-8-sig").dropna(subset=["Timestamp"])
    # Process PMV Data
    pmv_17 = process_pm_data(pd.read_excel('Winter_thermal_comfort_dataset/PMV_TSV.xlsx', sheet_name='2016 17').iloc[1:], '16')
    pmv_18 = process_pm_data(pd.read_excel('Winter_thermal_comfort_dataset/PMV_TSV.xlsx', sheet_name='2017 18').iloc[1:], '17')

    # Rename Columns for clarity in pmv data
    pmv_17, pmv_18 = preprocess_pmv_data(pmv_17, pmv_18)
    pmv_df = pd.concat([pmv_17, pmv_18])
    #print(pmv_df)
    # Extract data for each month
    months = ["11/2016", "12/2016", "01/2017", "02/2017", "03/2017", "12/2017", "01/2018", "02/2018", "03/2018"]

    for month in months:
        combine_monthly_data(bedroom_df, lr_df, pmv_df, month)

if __name__ == "__main__":
    main()
